# <b>DE Tools and their Weaknesses



### Dynamic Management Views (DMV's)

- Good: 
    - well documented by MS and blogs
    - easy to find scripts and tools that use them
- Bad: 
    - User written doco is wrong
    - misleading context
    - contents can reset unexpectedly
    - hit-or-miss coverage in Azure
    - keeps changing

- <b>sys.dm_db_index_usage_stats </b> (isnt really correct, stays around forever)
	- Shows # of executions where a plan included an operator (does NOT show if the operater was used, or how often it was accessed)
	- Number and last date of reads (seeks, scans, lookups)
	- Number and last date of last write (insert/update/deletes all called updates
	- Data is since startup or when the index was modified

- <b>sys.dm_db_index_operational_stats </b>  (very correct when it's correct, but not sure when its cleared)
	- Lower level, more transitory
	- Lock waits (page and row)
	- Access counts (doesnt distingush between full scans / range scans, or even range scans and seeks)
	- Data only persisted while objects metadata is in memory
	- No good way to tell when it was last cleared 

In [ ]:
USE StackOverflow;
GO
EXEC DropIndexes;
GO
CREATE INDEX IX_LastAccessDate ON dbo.Users(LastAccessDate);
GO
EXEC sp_BlitzIndex @SchemaName='dbo', @TableName='Users';
GO


Note: We are running BlitzIndex at a Table level, which tells us:

- What indexes we already have
- What their names are, columns are
- how many times they're being used

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.1.png" width=800>

We can see that we have 2 indexes at the moment - the Clustered Index (PK), and the non-clustered index we just created (on LastAccessDate)

Notice the <b>Usage Stats</b> (query plan DMV) and <b>Operational Stats</b> (touchy feeley DMV) columns. These comes from <b>sys.dm_db_index_usage_stats</b> and <b>sys.dm_db_index_operational_stats</b>

Now we run some queries an watch how they diverge. Look at the page reads for this query. Did it scan the whole index?

In [ ]:
SET STATISTICS IO ON;
GO
SELECT TOP 10 Id
FROM dbo.Users
ORDER BY LastAccessDate;
GO

/* Can you tell if it's a bad scan from the index DMVs? */
exec sp_BlitzIndex @SchemaName='dbo', @TableName='Users';
GO


According to the code above, this should result in a Table Scan (given no WHERE clause defined):

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.2.png" width=800>

And indeed it does, but is it a good scan or a bad scan? We can't actually tell... Some may complain that the index is only getting used for 'scans', which is the wrong diagnostic. 

We want to know is if the index was being used. Both Usage and Op Stats agree that the index was being used

Lets now reset the index usage statistics and run a new query

In [ ]:
/* Reset the index usage statistics. Man, I wish there was an easier way to do this in 2016. */
USE [master]
GO
ALTER DATABASE [StackOverflow] SET  OFFLINE WITH ROLLBACK IMMEDIATE;
GO
ALTER DATABASE [StackOverflow] SET ONLINE;
GO
USE StackOverflow;
GO

/* This plan is slightly different, it has a key lookup */
SELECT TOP 10 Id, Location
FROM dbo.Users
ORDER BY LastAccessDate;
GO


<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.3.png" width=600>

The SQL query engine does the following:
1. Do a table scan, finding the top 10 rows ordered by last access date
2. For every row it finds, do a key lookup to retrieve Location (location not included in our index)

Now we hover over the Key Lookup:

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.4.png" width=500></img>

Notice how the Number of Executions is 10. The plan only has the clustered index once, but we touched it 10 times. It's  clear that one of the DMV's shows the *number of plans* that got executed, the other shows the *number of times* the plan got executed (touchy feely)

    An Aside - Also notice that it read 200 / 10 lines from the NC Index. Because an index page is so dense, and that read-ahead is enabled, SQL engine continue to read from the index even though we only want the top 10 rows, just in case it needs them. It will stop reading rows once query execution is complete.

Now lets look back at the DMV's

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.5.png" width=850></img>

- For the non-clustered index, we only scanned it once (so they both agree)
- Different story for the clustered index. Usage stats reports that it was only scanned once. However, the Operational Stats DMV ('touchy feely') said we did this 10 times


In this case, op stats reports a higher value than the usage stats - however they are measuring different things, which is correct. But lets review a case where it doesn't quite show as we expect....



In [ ]:

/* This plan is slightly different, it has a key lookup - but it doesn't get executed. */
SELECT TOP 10 Id, Location
FROM dbo.Users
WHERE LastAccessDate > GETDATE()
ORDER BY LastAccessDate;
GO


This one looks for all those with a last access date in the future, which we expect not to return anything. However, SQL server does not know this when it first builds the query plan, so it will do the following:

1. Seek to the rows WHERE LastAccessDate > GETDATE()
2. Do a Key Lookup for the rows it finds

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.6.png" width=500></img>

The Index Seek tells us that it executed the seek, but it didnt find any rows

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.6.png" width=500></img>

Therefore, it never had to do the Key Lookup (Number of executions = 0)

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.7.png" width=850></img>

Here's where the index DMV's diverge... Usage stats tells us we did a read against the index, but Op Stats tells us we never touched it?

While they may not be the most reliable of measures, we're only looking to get a rough idea if our indexes are being used.

## Index DMV's - Takeaways

DMV's don't tell you that:
- "Scan" may not be the whole table (eg. Top X Selects)
- "Seek" might actually be the whole table (eg. WHERE clause using index with sub-optimal key ordering for that query)

sys.dm_db_index_usage_stats (Usage Stats)
- Show # of times an operator appeared in the query plan that was run
- The operateor may have been accessed many times, or not at all
- Reset by system restart, or by index rebuild if on buggy versions

sys.dm_db_index_operational_stats (Op Stats)
- Show number of times an operator was accessed
- Volatile, can be reset by memory pressure

The goal is just to know... "Are these indexes kinda getting used, or totally ignored?"

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.9.png" width=850></img>

In short, run from the higher, then zoom into each individual table. Prioritise by those that are generating the most noise, or causing users the most trouble

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.10.png" width=850></img>

<img src="C:\Users\hartleyg\Desktop\Training\SQL Server\Brent Ozar\Mastering Index Tuning\1.3.11.png" width=850></img>

## What we covered
The two SQL Server index usage views:
- Usage by plan: sys.dm_db_index_usage_stats
- Usage by index: sys.dm_db_index_operational_stats

Why they're not accurate as you might suspect:
- Seek doesnt mean one row
- Scan doesnt mean the whole table
- Reads dont mean the index was actually read
- 1 Write doesnt mean 1 row was updated
- They even reset at unusual times